# Differentiable bitonic sort

[Bitonic sorts](https://en.wikipedia.org/wiki/Bitonic_sorter) allow creation of sorting networks with a sequence of fixed conditional swapping operations. A sorting network implements  a map from $\mathbb{R}^n \rightarrow \mathbb{R}^n$, where $n=2^k$ (sorting networks for non-power-of-2 sizes are possible but not trickier).

<img src="BitonicSort1.svg.png">

*[Image: from Wikipedia, by user Bitonic, CC0](https://en.wikipedia.org/wiki/Bitonic_sorter#/media/File:BitonicSort1.svg)*

The sorting network for $2^k$ elements has $\frac{k(k-1)}{2}$ "layers" where parallel compare-and-swap operations are used to rearrange a $k$ element vector into sorted order.

### Differentiable compare-and-swap

If we define the `softmax(a,b)` function (not the traditional "softmax" used for classification!) as the continuous approximation to the `max(a,b)` function:

$$\text{softmax}(a,b) = \log(e^a + e^b) \approx \max(a,b).$$

We can then fairly obviously write `softmin(a,b)` as:

$$\text{softmin}(a,b) = -\log(e^{-a} + e^{-b}) \approx \min(a,b)$$

These aren't equal to max and min, but are relatively close, and differentiable. Note that we can now do a differentiable compare-and-swap operation:

$$\text{high} = \text{softmax}(a,b), \text{low} = \text{softmin}(a,b), \text{where } \text{low}\leq \text{high}$$

## Differentiable sorting

For each layer in the sorting network, we can split all of the pairwise comparison-and-swaps into left-hand and right-hand sides which can be done simultaneously. We can any write function that selects the relevant elements of the vector as a multiply with a binary matrix.

For each layer, we can derive two binary matrices $L \in \mathbb{R}^{k \times \frac{k}{2}}$ and $R \in \mathbb{R}^{k \times \frac{k}{2}}$ which select the elements to be compared for the left and right hands respectively. This will result in the comparison between two $\frac{k}{2}$ length vectors. We can also derive two matrices $L' \in \mathbb{R}^{\frac{k}{2} \times k}$ and $R' \in \mathbb{R}^{\frac{k}{2} \times k}$ which put the results of the compare-and-swap operation back into the right positions.

Then, each layer $i$ of the sorting process is just:
$${\bf x}_{i+1} = L'_i\text{softmin}(L_i{\bf x_i}, R_i{\bf x_i}) + R'_i\text{softmax}(L_i{\bf x_i}, R_i{\bf x_i})$$
$$ = L'_i\left(-\log\left(e^{-L_i{\bf x}_i} + e^{-R_i{\bf x}_i}\right)\right) +  R'_i\left(\log\left(e^{L_i{\bf x}_i} + e^{R_i{\bf x}_i}\right)\right)$$
which is clearly differentiable (though not very numerically stable -- the usable range of elements $x$ is quite limited in single float precision).

All that remains is to compute the matrices $L_i, R_i, L'_i, R'_i$ for each of the layers of the network. 

## Example

To sort four elements, we have a network like:

    0  1  2  3  
    ┕>>┙  │  │  
    │  │  ┕<<┙  
    ┕>>>>>┙  │  
    │  │  │  │  
    ┕>>┙  │  │  
    │  │  ┕>>┙  
    
This is equivalent to: 

    x[0], x[1] = cswap(x[0], x[1])
    x[3], x[2] = cswap(x[2], x[3])
    x[0], x[2] = cswap(x[0], x[2])
    x[0], x[1] = cswap(x[0], x[1])
    x[2], x[3] = cswap(x[2], x[3])
    
where `cswap(a,b) = (min(a,b), max(a,b))`

This can be written as three layers:
    
$$L_0 = \begin{bmatrix} 
    1 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    \end{bmatrix}
$$
    
$$R_0 = \begin{bmatrix} 
    0 & 1 & 0 & 0 \\
    0 & 0 & 0 & 1 \\
    \end{bmatrix}
$$

$$L'_0 = \begin{bmatrix} 
    1 & 0 \\
    0 & 0 \\
    0 & 0 \\
    0 & 1 \\
    \end{bmatrix}
$$
    
$$R'_0 = \begin{bmatrix} 
    0 & 0 \\     
    1 & 0 \\
    0 & 0 \\ 
    0 & 1 
    \end{bmatrix}
$$

$$L_1 = \begin{bmatrix} 
    1 & 0 & 0 & 0 \\
    0 & 1 & 0 & 0 \\
    \end{bmatrix}
$$
    
$$R_1 = \begin{bmatrix} 
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 \\
    \end{bmatrix}
$$

$$L'_1 = \begin{bmatrix} 
    1 & 0 \\
    0 & 1 \\
    0 & 0 \\
    0 & 0 \\
    \end{bmatrix}
$$
    
$$R'_1 = \begin{bmatrix} 
    0 & 0 \\     
    0 & 0 \\
    1 & 0 \\ 
    0 & 1 
    \end{bmatrix}
$$

$$L_2 = \begin{bmatrix} 
    1 & 0 & 0 & 0 \\
    0 & 0 & 1 & 0 \\
    \end{bmatrix}
$$
    
$$R_2 = \begin{bmatrix} 
    0 & 1 & 0 & 0 \\
    0 & 0 & 0 & 1 \\
    \end{bmatrix}
$$

$$L'_2 = \begin{bmatrix} 
    1 & 0 \\
    0 & 0 \\
    0 & 1 \\
    0 & 0 \\
    \end{bmatrix}
$$
    
$$R'_2 = \begin{bmatrix} 
    0 & 0 \\     
    1 & 0 \\
    0 & 0 \\ 
    0 & 1 
    \end{bmatrix}
$$


$$x_1 = L'_0(\text{softmin}(L_0x_0, R_0x_0)) + R'_0(\text{softmax}(Lx_0, Rx_0))$$
$$x_2 = L'_1(\text{softmin}(L_1x_1, R_1x_1)) + R'_1(\text{softmax}(L_1x_1, R_1x_1))$$
$$x_3 = L'_2(\text{softmin}(L_2x_2, R_2x_2)) + R'_2(\text{softmax}(L_2x_2, R_2x_2))$$


# Test functions

In [15]:
import numpy as np

def bitonic_network(n):
    """Check the computation of a bitonic network"""
    layers = int(np.log2(n))
    for layer in range(1, layers + 1):
        for sub in reversed(range(1, layer + 1)):
            for i in range(0, n, 2**sub):
                for j in range(2**(sub - 1)):
                    ix = i + j
                    a, b = ix, ix + (2**(sub - 1))
                    swap = "<" if (ix >> layer) & 1 else ">"
                    print(f"{a:>2}{swap}{b:<d}", end="\t")
            print()
        print("-" * n * 4)


# this should match the diagram at the top of the notebook
bitonic_network(16)

 0>1	 2<3	 4>5	 6<7	 8>9	10<11	12>13	14<15	
----------------------------------------------------------------
 0>2	 1>3	 4<6	 5<7	 8>10	 9>11	12<14	13<15	
 0>1	 2>3	 4<5	 6<7	 8>9	10>11	12<13	14<15	
----------------------------------------------------------------
 0>4	 1>5	 2>6	 3>7	 8<12	 9<13	10<14	11<15	
 0>2	 1>3	 4>6	 5>7	 8<10	 9<11	12<14	13<15	
 0>1	 2>3	 4>5	 6>7	 8<9	10<11	12<13	14<15	
----------------------------------------------------------------
 0>8	 1>9	 2>10	 3>11	 4>12	 5>13	 6>14	 7>15	
 0>4	 1>5	 2>6	 3>7	 8>12	 9>13	10>14	11>15	
 0>2	 1>3	 4>6	 5>7	 8>10	 9>11	12>14	13>15	
 0>1	 2>3	 4>5	 6>7	 8>9	10>11	12>13	14>15	
----------------------------------------------------------------


In [2]:
def pretty_bitonic_network(n):
    """Pretty print a bitonic network,
    to check the logic is correct"""
    layers = int(np.log2(n))
    # header
    for i in range(n):
        print(f"{i:<2d} ", end='')
    print()

    # layers
    for layer in range(1, layers + 1):
        for sub in reversed(range(1, layer + 1)):
            for i in range(0, n, 2**sub):
                for j in range(2**(sub - 1)):
                    ix = i + j
                    a, b = ix, ix + (2**(sub - 1))
                    way = "<" if (ix >> layer) & 1 else ">"

                    # this could be neater...
                    for i in range(n):
                        if i == b:
                            print("┙", end='')
                        elif i == a:
                            print("┕", end='')
                        elif not (a < i < b):
                            print("│", end='')
                        else:
                            print(way, end='')
                        if a <= i < b:
                            print(way * 2, end='')
                        else:
                            print(" " * 2, end='')
                    print()


pretty_bitonic_network(16)

0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 
┕>>┙  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  ┕<<┙  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  ┕>>┙  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  ┕<<┙  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  ┕>>┙  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  ┕<<┙  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  ┕>>┙  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  ┕<<┙  
┕>>>>>┙  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  ┕<<<<<┙  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  ┕>>>>>┙  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  │  │  │  │  │  │  ┕<<<<<┙  │  
│  │  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
┕>>┙  │  │  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  ┕>>┙  │  │  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  ┕<<┙  │  │  │  │  │  │  │  │  │  │  
│  │  │  │  │  │  ┕<

# Vectorised functions

In [4]:
import numpy as np


def softmax(a, b):
    """The softmaximum of softmax(a,b) = log(e^a + a^b)."""
    return np.log(np.exp(a) + np.exp(b))


def softmin(a, b):
    """
    Return the soft-minimum of a and b
    The soft-minimum can be derived directly from softmax(a,b).
    """
    return -softmax(-a, -b)


def softrank(a, b):
    """Return a,b in 'soft-sorted' order, with the smaller value first"""
    return softmin(a, b), softmax(a, b)

In [14]:
def bitonic_matrices(n):
    """Compute a set of bitonic sort matrices to sort a sequence of
    length n. n *must* be a power of 2.
    
    See: https://en.wikipedia.org/wiki/Bitonic_sorter
    
    Set k=log2(n).
    There will be k "layers", i=1, 2, ... k
    
    Each ith layer will have i sub-steps, so there are (k*(k+1)) / 2 sorting steps total.
    
    For each step, we compute 4 matrices. l and r are binary matrices of size (k/2, k) and
    inv_l and inv_r are matrices of size (k, k/2).
    
    l and r "interleave" the inputs into two k/2 size vectors. inv_l and inv_r "uninterleave" these two k/2 vectors
    back into two k sized vectors that can be summed to get the correct output.
                    
    The result is such that to apply any layer's sorting, we can perform:
    
    l, r, inv_l, inv_r = layer[j]
    a, b =  l @ y, r @ y                
    permuted = inv_l @ np.minimum(a, b) + inv_r @ np.maximum(a,b)
        
    Applying this operation for each layer in sequence sorts the input vector.
            
    """
    # number of outer layers
    layers = int(np.log2(n))
    matrices = []
    for layer in range(1, layers + 1):
        # we have 1..layer sub layers
        for sub in reversed(range(1, layer + 1)):
            l, r = np.zeros((n // 2, n)), np.zeros((n // 2, n))
            inv_l, inv_r = np.zeros((n, n // 2)), np.zeros((n, n // 2))
            out = 0
            for i in range(0, n, 2**sub):
                for j in range(2**(sub - 1)):
                    ix = i + j
                    a, b = ix, ix + (2**(sub - 1))
                    l[out, a] = 1
                    r[out, b] = 1
                    if (ix >> layer) & 1:
                        a, b = b, a
                    inv_l[a, out] = 1
                    inv_r[b, out] = 1
                    out += 1
            matrices.append((l, r, inv_l, inv_r))
    return matrices

import matrices
for matrix in bitonic_matrices(4):
    print(matrix)
            

(array([[1., 0., 0., 0.],
       [0., 0., 1., 0.]]), array([[0., 1., 0., 0.],
       [0., 0., 0., 1.]]), array([[1., 0.],
       [0., 0.],
       [0., 0.],
       [0., 1.]]), array([[0., 0.],
       [1., 0.],
       [0., 1.],
       [0., 0.]]))
(array([[1., 0., 0., 0.],
       [0., 1., 0., 0.]]), array([[0., 0., 1., 0.],
       [0., 0., 0., 1.]]), array([[1., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.],
       [1., 0.],
       [0., 1.]]))
(array([[1., 0., 0., 0.],
       [0., 0., 1., 0.]]), array([[0., 1., 0., 0.],
       [0., 0., 0., 1.]]), array([[1., 0.],
       [0., 0.],
       [0., 1.],
       [0., 0.]]), array([[0., 0.],
       [1., 0.],
       [0., 0.],
       [0., 1.]]))


In [6]:
def bisort(matrices, x):
    """
    Given a set of bitonic sort matrices generated by bitonic_matrices(n), sort 
    a sequence x of length n. Sorts exactly.
    """
    for l, r, map_l, map_r in matrices:
        a, b = l @ x, r @ x
        x = map_l @ np.minimum(a, b) + map_r @ np.maximum(a, b)
    return x


def diff_bisort(matrices, x):
    """
    Approximate differentiable sort. Takes a set of bitonic sort matrices generated by bitonic_matrices(n), sort 
    a sequence x of length n. Values will be distorted slightly but will be ordered.
    """
    for l, r, map_l, map_r in matrices:
        a, b = softrank(l @ x, r @ x)
        x = map_l @ a + map_r @ b
    return x

## Testing

In [21]:
# Test sorting
matrices = bitonic_matrices(8)

for i in range(10):
    # these should all be in sorted order
    test = np.random.randint(0, 200, 8)
    print(bisort(matrices, test))    

[53. 64. 66. 76. 83. 88. 95. 95.]
[ 22.  36.  76.  86.  97. 106. 174. 191.]
[ 18.  31.  41.  67.  67. 137. 159. 167.]
[  8.  10.  16. 100. 167. 171. 184. 198.]
[  2.  16.  48.  68.  80.  85.  87. 191.]
[  1.  25.  25.  45.  98. 119. 127. 191.]
[ 33.  58.  60.  61. 129. 137. 152. 183.]
[ 14.  27.  47.  81. 107. 107. 131. 153.]
[ 41.  57.  67.  87.  99. 117. 151. 189.]
[ 13.  22.  47.  76.  99. 109. 124. 160.]


In [7]:
for i in range(1, 11):
    k = 2**i
    matrices = bitonic_matrices(k)
    print(f"Testing sorting for {k} elements")
    for j in range(100):
        test = np.random.randint(0, 200, k)
        assert (np.allclose(bisort(matrices, test), np.sort(test)))

Testing sorting for 2 elements
Testing sorting for 4 elements
Testing sorting for 8 elements
Testing sorting for 16 elements
Testing sorting for 32 elements
Testing sorting for 64 elements
Testing sorting for 128 elements
Testing sorting for 256 elements
Testing sorting for 512 elements
Testing sorting for 1024 elements


## Differentiable sorting test

In [20]:
# Differentiable sorting 
np.set_printoptions(precision=2)
matrices = bitonic_matrices(8) 
def neat_vec(n):
    return "\t".join([f"{x:.2f}" for x in n])

for i in range(10):
    test = np.random.randint(-200,200,8)
    print("Differentiable", neat_vec(diff_bisort(matrices, test)))
    print("Exact sorting ", neat_vec(bisort(matrices, test)))
    print()

Differentiable -180.01	-175.01	-170.98	-142.00	-103.00	-27.00	18.00	62.00
Exact sorting  -180.00	-175.00	-171.00	-142.00	-103.00	-27.00	18.00	62.00

Differentiable -161.00	-104.00	-25.00	-19.00	31.00	92.00	105.00	162.00
Exact sorting  -161.00	-104.00	-25.00	-19.00	31.00	92.00	105.00	162.00

Differentiable -182.00	-151.00	-127.00	-76.31	-74.69	46.00	92.00	185.00
Exact sorting  -182.00	-151.00	-127.00	-76.00	-75.00	46.00	92.00	185.00

Differentiable -198.00	-170.00	-120.00	17.00	140.00	155.00	170.00	183.00
Exact sorting  -198.00	-170.00	-120.00	17.00	140.00	155.00	170.00	183.00

Differentiable -193.00	-101.00	-67.00	56.00	76.00	83.00	176.00	193.00
Exact sorting  -193.00	-101.00	-67.00	56.00	76.00	83.00	176.00	193.00

Differentiable -187.00	-58.00	-46.02	-41.98	-8.00	49.00	84.98	90.02
Exact sorting  -187.00	-58.00	-46.00	-42.00	-8.00	49.00	85.00	90.00

Differentiable -115.13	-111.87	-84.00	-36.00	58.00	123.00	135.00	166.00
Exact sorting  -115.00	-112.00	-84.00	-36.00	58.00	123.00	135.00	1

# PyTorch example
We can verify that this is both parallelisable on the GPU and fully differentiable.

In [9]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device:', device)

ModuleNotFoundError: No module named 'torch'

In [ ]:
matrices = bitonic_matrices(16)
torch_matrices = [[torch.from_numpy(matrix).float().to(device) for matrix in matrix_set] for matrix_set in matrices]

In [ ]:
# override softmax to use torch tensors
def softmax(a, b):
    """The softmaximum of softmax(a,b) = log(e^a + e^b)."""
    return torch.log(torch.exp(a) + torch.exp(b))

In [ ]:
test_input = np.random.normal(0, 5, 16)
var_test_input = Variable(torch.from_numpy(test_input).float().to(device),
                          requires_grad=True)
result = diff_bisort(torch_matrices, var_test_input)

# compute the Jacobian of the sorting function, to show we can differentiate through the
# sorting function
jac = []
for i in range(len(result)):
    jac.append(
        torch.autograd.grad(result[i], var_test_input, retain_graph=True)[0])

# 16 x 16 jacobian of the sorting matrix
print(torch.stack(jac))